In [5]:
pip install python-telegram-bot openpyxl pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install PyTelegramBotAPI openpyxl pandas

     ---------------------------------------- 0.0/48.3 kB ? eta -:--:--
     ---------------------------------------- 48.3/48.3 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/287.4 kB ? eta -:--:--
   ---------------------------------------  286.7/287.4 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 287.4/287.4 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
   ---------------------------------------- 105.4/105.4 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/128.7 kB ? eta -:--:--
   ---------------------------------------- 128.7/128.7 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, ContextTypes
import pandas as pd
import os

# Загружаем Excel
df = pd.read_excel('products.xlsx')

# Получаем уникальные категории
categories = df['Категория'].unique()

# Приветствие
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = [
        [InlineKeyboardButton("Купить", callback_data='buy')],
        [InlineKeyboardButton("Оставить отзыв", callback_data='review')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Добро пожаловать в наш магазин! Что вы хотите сделать?", reply_markup=reply_markup)

# Обработка нажатий
async def button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    if query.data == 'buy':
        keyboard = [[InlineKeyboardButton(cat, callback_data=f"cat_{cat}")] for cat in categories]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await query.edit_message_text("Выберите категорию:", reply_markup=reply_markup)
    
    elif query.data.startswith('cat_'):
        category = query.data.split('_', 1)[1]
        products = df[df['Категория'] == category]
        for _, row in products.iterrows():
            text = f"{row['Название']}\nЦена: {row['Цена']} руб."
            photo_path = os.path.join('photos', row['Фото'])  # Путь к фото
            if os.path.exists(photo_path):
                with open(photo_path, 'rb') as photo:
                    await query.message.reply_photo(photo=photo, caption=text)
            else:
                await query.message.reply_text(text)

    elif query.data == 'review':
        await query.edit_message_text("Вы можете оставить отзыв, написав нам сюда.")

# Главная функция
def main():
    TOKEN = "7713900787:AAHuEPpjV4Kgk5hPEA_dMTdQFOo9Z2yL7CM"  # <-- замените на ваш токен
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CallbackQueryHandler(button))

    application.run_polling()
    if __name__ == '__main__':
        main()

In [10]:
from telegram.ext import Application

TOKEN = "7713900787:AAHuEPpjV4Kgk5hPEA_dMTdQFOo9Z2yL7CM"

application = Application.builder().token(TOKEN).build()
application.add_handler(CommandHandler("start", start))
application.add_handler(CallbackQueryHandler(button))

await application.initialize()
await application.start()
await application.updater.start_polling()


<Queue at 0x126ecdf2010 maxsize=0 _getters[1]>